<a href="https://colab.research.google.com/github/Isha1145/Crop-yield/blob/main/sugarcane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Sugarcane Data

In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from collections import namedtuple, deque
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

df = pd.read_csv("sugarcane_data.csv")  # Your dataset file
# Normalize or scale if needed
features = df[['rainfall', 'temp', 'ndvi']].values
target = df['yield'].values


FileNotFoundError: [Errno 2] No such file or directory: 'sugarcane_data.csv'

Define Environment Interface

In [ ]:
class SugarcaneEnv:
    def __init__(self, features, target):
        self.features = features
        self.target = target
        self.current = 0
        self.max_steps = len(features)

    def reset(self):
        self.current = 0
        return self.features[self.current]

    def step(self, action):
        # Custom reward logic: Example — reward = inverse error between predicted and true yield
        actual_yield = self.target[self.current]
        predicted_yield = action  # action will be scalar float from network
        reward = -abs(predicted_yield - actual_yield)

        self.current += 1
        done = self.current >= self.max_steps
        next_state = self.features[self.current] if not done else np.zeros_like(self.features[0])
        return next_state, reward, done


Define Agent & Q-Network

In [ ]:
class QNetwork(nn.Module):
    def __init__(self, input_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 1)  # Predict yield (regression)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class Agent:
    def __init__(self, state_size):
        self.qnetwork = QNetwork(state_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork.parameters(), lr=1e-3)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        with torch.no_grad():
            return self.qnetwork(state).cpu().numpy()[0][0]

    def train(self, experiences):
        states, actions, rewards = experiences
        states = torch.tensor(states).float().to(device)
        actions = torch.tensor(actions).float().unsqueeze(1).to(device)
        rewards = torch.tensor(rewards).float().unsqueeze(1).to(device)

        pred = self.qnetwork(states)
        loss = F.mse_loss(pred, rewards)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


Training Loop

In [ ]:
env = SugarcaneEnv(features, target)
agent = Agent(state_size=features.shape[1])

scores = []
for episode in range(500):
    state = env.reset()
    total_reward = 0
    memory = []

    for _ in range(env.max_steps):
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        memory.append((state, action, reward))
        state = next_state
        total_reward += reward
        if done:
            break

    if len(memory) > 0:
        states, actions, rewards = zip(*memory)
        agent.train((states, actions, rewards))

    scores.append(total_reward)
    if episode % 50 == 0:
        print(f"Episode {episode}, Total Reward: {total_reward:.2f}")

# Plot training rewards
plt.plot(scores)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Training Progress on Sugarcane Dataset')
plt.show()
